In [3]:
import pandas as pd
import numpy as np

import torch
from torch.utils.data import Dataset

In [3]:
ratings_df = pd.read_csv("rating_gener_writer_df_100.csv")

In [4]:
ratings_df.head(10)

,user,item,rating,1,2,3,4,5,6,7,...,10,11,12,13,14,15,16,17,18,writer
0,11.0,4643.0,1.0,1,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,536
1,189.0,4643.0,1.0,1,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,536
2,294.0,4643.0,1.0,1,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,536
3,383.0,4643.0,1.0,1,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,536
4,421.0,4643.0,1.0,1,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,536
5,422.0,4643.0,1.0,1,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,536
6,609.0,4643.0,1.0,1,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,536
7,829.0,4643.0,1.0,1,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,536
8,951.0,4643.0,1.0,1,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,536
9,1068.0,4643.0,1.0,1,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,536


In [7]:
col_len = list() #[31360, 6807, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2027]

for col in ratings_df.columns:
    if col == "rating":
        continue
    category = ratings_df[f"{col}"].astype('category')
    col_len.append(len(ratings_df[f"{col}"].astype('category').cat.categories))
    ratings_df[f"{col}"] = category.cat.codes

In [16]:
ratings_df

,user,item,rating,1,2,3,4,5,6,7,...,10,11,12,13,14,15,16,17,18,writer
0,0,2505,1.0,1,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,536
1,39,2505,1.0,1,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,536
2,66,2505,1.0,1,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,536
3,85,2505,1.0,1,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,536
4,95,2505,1.0,1,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,536
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8290466,31299,6630,0.0,1,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1713
8290467,31312,6630,0.0,1,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1713
8290468,31316,6630,0.0,1,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1713
8290469,31325,6630,0.0,1,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1713


In [17]:
train_X = ratings_df.loc[:, ratings_df.columns != 'rating']
train_y = ratings_df.loc[:,'rating']

In [24]:
len(np.array(train_y))

8290471

In [6]:
col_len = [31360, 6807, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2027]
offsets = np.concatenate([[0], np.cumsum(col_len)[:-1]])


array([    0, 31360, 38167, 38169, 38171, 38173, 38175, 38177, 38179,
       38181, 38183, 38185, 38187, 38189, 38191, 38193, 38195, 38197,
       38199, 38201, 38203])

In [21]:
torch.tensor(np.array(train_X)[0] + offsets)

tensor([    0, 33865, 38168, 38170, 38171, 38173, 38175, 38177, 38179, 38182,
        38183, 38185, 38187, 38189, 38191, 38193, 38196, 38197, 38199, 38201,
        38739])

In [9]:
class TrainDataset(Dataset):
    def __init__(self):
        
        rating_df = pd.read_csv("rating_gener_writer_df_100.csv")

        self.offsets = list()
        col_len = list() #[31360, 6807, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2027]

        for col in rating_df.columns:
            if col == "rating":
                continue
            category = rating_df[f"{col}"].astype('category')
            col_len.append(len(rating_df[f"{col}"].astype('category').cat.categories))
            rating_df[f"{col}"] = category.cat.codes

        self.offsets = np.concatenate([[0], np.cumsum(col_len)[:-1]])
        self.input_dims = col_len

        self.train_X = np.array(rating_df.loc[:, rating_df.columns != 'rating'])
        self.train_y = np.array(rating_df.loc[:,'rating'])

    def __getitem__(self, index):
        X = self.train_X[index] + self.offsets
        y = self.train_y[index]
        return torch.tensor(X), torch.tensor(y)

    def __len__(self):
        return len(self.train_y)

In [4]:
class InferenceDataset(Dataset):
    def __init__(self):
        
        inference_df = pd.read_csv("inference_base.csv") 
        inference_df.sort_values(by="user",axis = 0, inplace = True)

        self.offsets = list()
        col_len = list() #[31360, 6807, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2027]

        for col in inference_df.columns:
            category = inference_df[f"{col}"].astype('category')
            col_len.append(len(inference_df[f"{col}"].astype('category').cat.categories))

        self.offsets = np.concatenate([[0], np.cumsum(col_len)[:-1]])
        self.input_dims = col_len

        self.X = np.array(inference_df)

    def __getitem__(self, index):
        X = self.X[index] + self.offsets
        return torch.tensor(X)

    def __len__(self):
        return len(self.X)

In [10]:
train_dataset = TrainDataset()

In [11]:
train_dataset[0]

(tensor([    0, 33865, 38168, 38170, 38171, 38173, 38175, 38177, 38179, 38182,
         38183, 38185, 38187, 38189, 38191, 38193, 38196, 38197, 38199, 38201,
         38739]),
 tensor(1., dtype=torch.float64))

In [12]:
inference_dataset = InferenceDataset()
inference_dataset[0]

tensor([    0, 31361, 32526, 32529, 32530, 32533, 32534, 32536, 32538, 32540,
        32543, 32544, 32546, 32548, 32550, 32552, 32554, 32556, 32558, 32560,
        33664])

- user column name to "userId"
- item column name to "movieId"
- add rating to 1.0
- time to "timestamp"

In [36]:
item_ids = train_ratings_df['item'].unique() # 6807
user_ids = train_ratings_df['user'].unique() # 31360

#-- Re-index user, item
# CAUTION : user starts with index 0 (1 ~ 31360)
#           item starts with index 1 (1 ~ 6807) 
item2idx = pd.Series(data=np.arange(len(item_ids)) + 1, index=item_ids)
user2idx = pd.Series(data=np.arange(len(user_ids)) + 1, index=user_ids)

# user2idx [original_user] = reIndexed user
# item2idx [original item] = reIndexed item
# user_ids [reIndexed user - 1] = original_user
# item_idx [reIndexed item - 1] = original_item

In [37]:
train_ratings_df = pd.merge(train_ratings_df, pd.DataFrame({'item': item_ids, 'movieId': item2idx[item_ids].values}), on='item', how='inner')
train_ratings_df = pd.merge(train_ratings_df, pd.DataFrame({'user': user_ids, 'userId': user2idx[user_ids].values}), on='user', how='inner')
train_ratings_df.sort_values(['userId', 'time'], inplace=True)
del train_ratings_df['item'], train_ratings_df['user']

In [38]:
train_ratings_df

,time,movieId,userId
0,1230782529,1,1
1,1230782534,2,1
2,1230782539,3,1
3,1230782542,4,1
4,1230782563,5,1
...,...,...,...
368315,1260209449,424,31360
368422,1260209482,1492,31360
368301,1260209720,332,31360
368360,1260209726,734,31360


In [39]:
train_ratings_df["rating"] = [1.0] * len(train_ratings_df["time"])

In [40]:
train_ratings_df.columns=["timestamp", "movieId", "userId", "rating"]

In [41]:
train_ratings_df = train_ratings_df[["userId", "movieId", "rating", "timestamp"]]
train_ratings_df

,userId,movieId,rating,timestamp
0,1,1,1.0,1230782529
1,1,2,1.0,1230782534
2,1,3,1.0,1230782539
3,1,4,1.0,1230782542
4,1,5,1.0,1230782563
...,...,...,...,...
368315,31360,424,1.0,1260209449
368422,31360,1492,1.0,1260209482
368301,31360,332,1.0,1260209720
368360,31360,734,1.0,1260209726


In [34]:
train_ratings_df.to_csv("./ratings.csv", index=False)